# Polymarket API Python

**Resources:**
- [Polymarket API Docs](https://docs.polymarket.com/)
- [Gamma API (Market Data)](https://gamma-api.polymarket.com/)
- [Data API (User Data)](https://data-api.polymarket.com/)
- [py-clob-client (3rd Party using Official SDK)](https://github.com/Polymarket/py-clob-client)

## 1. Setup

In [ ]:
# pip install py-clob-client requests

In [ ]:
import requests
import time
import json
from pprint import pprint

from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, MarketOrderArgs, OrderType, OpenOrderParams, BalanceAllowanceParams, AssetType
from py_clob_client.order_builder.constants import BUY, SELL

GAMMA_API = "https://gamma-api.polymarket.com"
DATA_API = "https://data-api.polymarket.com"
CLOB_API = "https://clob.polymarket.com"

## 2. Market Discovery

In [ ]:
# Fetch active markets sorted by volume
response = requests.get(
    f"{GAMMA_API}/markets",
    params={
        "limit": 10,
        "active": True,
        "closed": False,
        "order": "volume24hr",
        "ascending": False
    }
)
markets = response.json()
print(f"Found {len(markets)} markets\n")

In [ ]:
for m in markets[:5]:
    print(f"Question: {m['question']}")
    print(f"  Volume 24h: ${m.get('volume24hr', 0):,.0f}")
    print(f"  Liquidity: ${m.get('liquidityNum', 0):,.0f}")
    print(f"  Prices: {m.get('outcomePrices', 'N/A')}")
    print()

## 3. Market Deep Dive

In [ ]:
market = markets[1]
market

In [ ]:
print(f"Market: {market['question']}")
print(f"End Date: {market['endDate']}")
print(f"Condition ID: {market['conditionId']}")

In [ ]:
clob_token_ids = market.get('clobTokenIds')
clob_token_ids = json.loads(clob_token_ids)
print(f"Token IDs: {clob_token_ids}")

if len(clob_token_ids) >= 2:
    yes_token_id = clob_token_ids[0]
    no_token_id = clob_token_ids[1]
    print(f"YES token: {yes_token_id}")
    print(f"NO token: {no_token_id}")

## 4. Order Book Analysis

In [ ]:
# Read-only client for order book
client = ClobClient(CLOB_API)

In [ ]:
# Fetch order book for YES token
if yes_token_id:
    book = client.get_order_book(yes_token_id)
    
    sorted_bids = sorted(book.bids, key=lambda x: float(x.price), reverse=True)
    sorted_asks = sorted(book.asks, key=lambda x: float(x.price), reverse=False)
    
    print("=== YES Token Order Book ===")
    print(f"\nTop 5 Asks (sell orders):")
    for ask in sorted_asks[:5]:
        print(f"  Price: {ask.price} | Size: {ask.size}")
        
    print(f"\nTop 5 Bids (buy orders):")
    for bid in sorted_bids[:5]:
        print(f"  Price: {bid.price} | Size: {bid.size}")


In [ ]:
# Get midpoint, prices, and spread
mid = client.get_midpoint(yes_token_id)
buy_price = client.get_price(yes_token_id, side="BUY")
sell_price = client.get_price(yes_token_id, side="SELL")
spread = client.get_spread(yes_token_id)

print(f"Midpoint: {mid['mid']}")
print(f"Best ask (buy at): {buy_price['price']}")
print(f"Best bid (sell at): {sell_price['price']}")
print(f"Spread: {spread['spread']}")

## 5. Authentication

In [ ]:
# Your credentials (keep these secret!)
FUNDER_ADDRESS = "your-wallet-address"  # For proxy wallets
PRIVATE_KEY = "your-private-key-here"  # Never commit this

# Signature types:
# 0 = EOA (MetaMask, hardware wallet)
# 1 = Email/Magic wallet
# 2 = Browser wallet proxy
SIGNATURE_TYPE = 1

In [ ]:
# Initialize authenticated client
auth_client = ClobClient(
    CLOB_API,
    key=PRIVATE_KEY,
    chain_id=137,
    signature_type=SIGNATURE_TYPE,
    funder=FUNDER_ADDRESS
)

# Get & Set API credentials
creds = auth_client.derive_api_key()
auth_client.set_api_creds(creds)

In [ ]:
balance = auth_client.get_balance_allowance(BalanceAllowanceParams(asset_type=AssetType.COLLATERAL))
usdc_balance = int(balance['balance']) / 1e6
print(f"USDC Balance: ${usdc_balance:.2f}")

## 6. Place a Market Order

In [ ]:
# Create a market order: Spend $5 on YES shares at market price
market_order = MarketOrderArgs(
    token_id=yes_token_id,
    amount=5.0,  # Dollar amount to spend
    side=BUY,
    order_type=OrderType.FOK  # Fill-or-Kill
)

signed_market_order = auth_client.create_market_order(market_order)
print("Market order signed!")

In [ ]:
# Execute market order
response = auth_client.post_order(signed_market_order, OrderType.FOK)
print("Market order executed!")
pprint(response)

## 7. Place a Limit Order

In [ ]:
# Create a limit order: Buy 10 YES shares at $0.50
limit_order = OrderArgs(
    token_id=yes_token_id,
    price=0.001,  # Price per share
    size=10.0,   # Number of shares
    side=BUY
)

# Sign the order
signed_order = auth_client.create_order(limit_order)
print("Order signed!")

In [ ]:
# Post the order (GTC = Good Till Cancelled)
response = auth_client.post_order(signed_order, OrderType.GTC)
print(f"Order placed!")
pprint(response)

## 8. Open Orders

In [ ]:
# Get your open orders
open_orders = auth_client.get_orders(OpenOrderParams())
print(f"Open orders: {len(open_orders)}")

for order in open_orders[:5]:
    print(f"  ID: {order['id'][:20]}...")
    print(f"  Side: {order['side']} | Price: {order['price']} | Size: {order['original_size']}")
    print()

## 9. Cancel Orders

In [ ]:
# Cancel a specific order
open_orders = auth_client.get_orders(OpenOrderParams())
if open_orders:
    order_id = open_orders[0]["id"]
    result = auth_client.cancel(order_id)
    print(f"Cancelled order: {order_id[:20]}...")
    pprint(result)

In [ ]:
# Cancel all orders
result = auth_client.cancel_all()
print("All orders cancelled!")
pprint(result)

## 10. BONUS 1: Price Tracker

In [ ]:
def track_price(token_id, duration_seconds=30, interval=5):
    """Track price changes in real-time."""
    print(f"Tracking price for {duration_seconds}s...\n")

    client = ClobClient(CLOB_API)
    start_time = time.time()
    prices = []

    while time.time() - start_time < duration_seconds:
        mid = client.get_midpoint(token_id)
        mid_price = float(mid['mid'])
        timestamp = time.strftime("%H:%M:%S")
        prices.append(mid_price)

        change = ""
        if len(prices) > 1:
            diff = prices[-1] - prices[-2]
            change = f" ({'+' if diff >= 0 else ''}{diff:.4f})"

        print(f"[{timestamp}] Price: {mid_price}{change}")
        time.sleep(interval)

    print(f"\nTotal change: {prices[-1] - prices[0]:.4f}")
    return prices

In [ ]:
# Run the tracker
prices = track_price(yes_token_id, duration_seconds=10, interval=1)

## 11. BONUS 2: Address Position Tracker

In [ ]:
def get_user_positions(wallet_address):
    """Get a user's current positions."""
    url = f"{DATA_API}/positions"
    params = {"user": wallet_address}
    response = requests.get(url, params=params)
    return response.json()


In [ ]:
address = ""
positions = get_user_positions(address)
positions